In [ ]:
import os
import requests
import json
import base64
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import stability_sdk.client as client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation


In [ ]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', "OpenAI API Key not set")
news_api_key = os.getenv('NEWS_API_KEY', "News API Key not set")
stability_api_key = os.getenv('STABILITY_API_KEY', "Image API Key not set")


MODEL = 'gpt-4o-mini'
openai = OpenAI()


In [ ]:
system_message = "You are a helpful assistant for FindingLatestNewsAI."
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
BASE_URL = "https://newsapi.org/v2/everything"

def get_news(keyword):
    print(f"Tool get_news called for keyword: {keyword}")
    params = {
        "q": keyword.lower(),
        "apiKey": news_api_key,
        "language": "en",
        "pageSize": 5
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    print()

    if "articles" not in data:
        return "Please check API setting"

    articles = data["articles"]
    news_list = []

    if not articles:
        return f"Sorry I can not find {keyword} related news. Please try another keyword"

    result_text = f"**{keyword} related news**\n\n"
    for article in articles:
        title = article['title']
        source = article['source']['name']
        url = article['url']

        result_text += f"{article['title']} ({article['source']['name']})\n"
        result_text += f"Here's the news ({article['url']})\n\n"

        news_list.append({"title": title, "source": source, "url": url})

    return json.dumps(news_list, ensure_ascii=False)
    

In [ ]:
news_function = {
    "name": "get_news",
    "description": "Get the latest news from keyword",
    "parameters": {
        "type": "object",
        "properties": {
            "keyword": {
                "type": "string",
                "description": "user would type the keyword to look for the news. For example, 'technology','AI', 'Bitcoin' or 'I would like to know the latest news for AI' "
            },
        },
        "required":["keyword"],
        "additionalProperties": False
    }
}


In [ ]:
tools = [{"type": "function", "function": news_function}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    keyword = arguments.get('keyword')
    news = get_news(keyword)
    response = {
        "role": "tool",
        "content": json.dumps({"keyword": keyword, "news": news}),
        "tool_call_id": tool_call.id
    }

    return response, keyword

In [ ]:
import io

def generate_image(keyword):  
    stability_api = client.StabilityInference(
        key=stability_api_key,
        verbose=True,
        engine="stable-diffusion-xl-1024-v1-0" 
    )

    answers = stability_api.generate(
        prompt=keyword,  
        seed=None,  
        steps=30,  
        cfg_scale=7.0,  
        width=512,  
        height=512,  
        samples=1  
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.type == generation.ARTIFACT_IMAGE:
                image = Image.open(io.BytesIO(artifact.binary))
                display(image)
                return image

In [ ]:
def extract_keywords(news_text):
    import openai
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Extract the most important 2-3 words that best describe this news for image generation."},
                  {"role": "user", "content": news_text}]
    )
    return response.choices[0].message.content.strip()

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    keyword = arguments.get('keyword')
    news_json = get_news(keyword)
    news_data = json.loads(news_json)  # 轉換回 Python 字典列表
    if not news_data or len(news_data) == 0:
        return {"role": "tool", "content": f"No news found for '{keyword}'"}, keyword
    first_news_title = news_data[0]["title"]
    keyword_for_image = extract_keywords(first_news_title)
    image = generate_image(keyword_for_image)
    response = {
        "role": "tool",
        "content": json.dumps({"keyword": keyword, "news": news}),
        "tool_call_id": tool_call.id
    }

    return response, keyword

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())


    display(Audio(output_filename, autoplay=True))

# talker("Well, hi there")

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, keyword = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = generate_image(keyword)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]


    talker(reply)
    
    return history, image

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def chat_and_image(history):
        history, image = chat(history)
        return history, image
    
    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat_and_image, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch()